# OLD CODE

## **RERUN DLL WITH MASK**

In [ ]:
# import numpy as np

# Option_Model = "DEM_2_LDD_PYFLWDIR"  # <"DEM_2_LDD_HYDROMT">,<DEM_2_LDD_PYFLWDIR>

# Path_Input_Catchment = os.path.join(Path_Root, Path_Python, Filename_Output_Subcatchment)

# with rasterio.open(Path_Input_Catchment, "r") as src:
#     Catchment = src.read(1)
#     nodata = src.nodata
#     transform = src.transform
#     crs = src.crs
#     latlon = src.crs.is_geographic
#     prof = src.profile

# Path_Input_Dem = os.path.join(Path_Root, Path_Julia, Filename_Input_DemJulia)

# with rasterio.open(Path_Input_Dem, "r") as src:
#     Dem = src.read(1)
#     nodata = src.nodata
#     transform = src.transform
#     crs = src.crs
#     latlon = src.crs.is_geographic
#     prof = src.profile

# # Masking
# Nx, Ny = np.shape(Dem)

# Dem_Mask = np.ma.empty(shape=(Nx, Ny))
# for iX in range(Nx):
#     for iY in range(Ny):
#         if Catchment[iX, iY] > 0:
#             Dem_Mask[iX, iY] = Dem[iX, iY]
#         else:
#             Dem_Mask[iX, iY] = np.nan

# # Writting masked
# Path_Output_DemMasked = os.path.join(Path_Root, Path_Julia, Filename_Output_DemCorrectedMasked)
# prof.update(dtype=rasterio.float64, nodata=np.nan)
# with rasterio.open(Path_Output_DemMasked, "w", **prof) as src:
#     src.write(Dem_Mask, 1)

# # Rerunning the Ldd with the masked DEM
# Path_Output_Ldd = os.path.join(Path_Root, Path_Python, Filename_Output_Ldd)

# if Option_Model == "DEM_2_LDD_HYDROMT":
#     Ldd_Pyflwdir = DEM_2_LDD_HYDROMT(
#         Path_Output_DemMasked,
#         Path_Output_Ldd,
#         Param_PitMaxDepth=Param_PitMaxDepth,
#         Param_RiverDepth=Param_RiverDepth,
#         Param_NoData=Param_NoData,
#         outlets=Param_OutletMethod,
#         River=River_Geopandas,
#         Idxs_pit=None,
#     )
#     PLOT_MAPS(Ldd_Pyflwdir, "Ldd", 0, 10)

# else:
#     Ldd_Pyflwdir, Ldd_Array = DEM_2_LDD_PYFLWDIR(
#         Path_Output_DemMasked, Path_Output_Ldd, Param_PitMaxDepth=Param_PitMaxDepth, Gauges=Param_OutletMethod)

#     PLOT_MAPS(Ldd_Array, "Ldd", 0, 10)


In [ ]:
gdf_out = gpd.GeoSeries(gpd.points_from_xy(*flw.xy(idxs_out), crs=4326))

# flow paths return the list of linear indices
xy = ([8.92, 5.55, 8.50], [50.28, 49.80, 47.3])
flowpaths, dists = flw.path(xy=xy, max_length=400e3, unit="m")
# note that all distances are nearly at the threshold
dists / 1e3

coords_to_idxs(xs, ys, transform, shape, op=np.floor, precision=None):

FlwdirRaster.index(xs, ys, **kwargs)
FlwdirRaster.xy(idxs, **kwargs)

FlwdirRaster.add_pits(idxs=None, xy=None, streams=None)[source]
Add pits the flow direction raster. If streams is given, the pits are snapped to the first downstream True cell.

Parameters
:
idxs (array_like, optional) – linear indices of pits, by default is None.

xy (tuple of array_like of float, optional) – x, y coordinates of pits, by default is None.

streams (2D array of bool, optional) – 2D raster with cells flagged ‘True’ at stream cells, only used in combination with idx or xy, by default None.

feats = flw.streams(min_sto=4)
gdf = gpd.GeoDataFrame.from_features(feats, crs=crs)


SyntaxError: invalid character '–' (U+2013) (3714598091.py, line 19)

In [ ]:
import rasterio
import numpy as np

with rasterio.open(r'F:\projects\ordos_flood\Naeem\DEM\Taolimiao_500m/flwdir0.tif') as src:       # input file path
   image = src.read(1)  # read the first band
   mask = src.dataset_mask()  # get the mask, non-zero means active pixel

   # for edge detecting
   edges = mask == 0
   dilated = np.logical_or(edges, np.roll(edges, 1, axis=0))
   dilated = np.logical_or(dilated, np.roll(edges, -1, axis=0))
   dilated = np.logical_or(dilated, np.roll(edges, 1, axis=1))
   dilated = np.logical_or(dilated, np.roll(edges, -1, axis=1))

   # set pixel value in the edge to zero
   image[dilated] = 0

   # save the revised image
   profile = src.profile
   with rasterio.open(r'F:\projects\ordos_flood\Naeem\DEM\Taolimiao_500m/flwdir.tif', 'w', **profile) as dst:    # output file path
      dst.write(image, 1)

#########################################


In [ ]:
import os as os

Path_Input_Dem = os.path.join(Path_Root, Path_Julia, Filename_Input_DemJulia)
Path_Output_Ldd_Basins = os.path.join(
   Path_Root, Path_Python, Filename_Output_Ldd_Basins)
Path_Input_River_Geojson = os.path.join(
   Path_Root, Path_Gis, Filename_Input_RiverShp)

Ldd_Basins = []
if Flag_BurnRiver:
   River_Geopandas = geopandas.read_file(Path_Input_River_Geojson)
   Ldd_Basins = DEM_2_LDD_HYDROMT(
      Path_Input_Dem,
      Path_Output_Ldd_Basins,
      Param_PitMaxDepth=Param_PitMaxDepth,
      Param_RiverDepth=Param_RiverDepth,
      Param_NoData=Param_NoData,
      outlets="edge",
      River=River_Geopandas,
      Idxs_pit=None,
      Flag_BurnRiver=Flag_BurnRiver)
else:
   Ldd_Basins = DEM_2_LDD_HYDROMT(
      Path_Input_Dem,
      Path_Output_Ldd_Basins,
      Param_PitMaxDepth=Param_PitMaxDepth,
      Param_RiverDepth=Param_RiverDepth,
      Param_NoData=Param_NoData,
      outlets="edge",
      River=[],
      Idxs_pit=None,
      Flag_BurnRiver=Flag_BurnRiver)

PLOT_MAPS(Ldd_Basins, "Ldd", 1, 9)

# elif Param_LddMethod =="Pyflwdir":
#     print("==Ldd Pyflwdir==")
#     Ldd_Basins, Ldd_Basins_FlwdirRaster = DEM_2_LDD_PYFLWDIR(
#         Path_Input_Dem,
#         Path_Output_Ldd_Basins,
#         Param_NoData = Param_NoData,
#         outlets      = "edge",
#         River        = [],
#         Idxs_pit     = None)

#     iiPits_Vector = Ldd_Basins_FlwdirRaster.idxs_pit
#     print(iiPits_Vector)

#     if not(Ldd_Basins_FlwdirRaster.isvalid):
#         print("========= Ldd_Basins_FlwdirRaster not valid ========")


#     PLOT_MAPS(Ldd_Basins, "Ldd", 1, 9)

# elif Param_LddMethod =="Pcraster":
#     print("==Ldd Pcraster==")
#     Ldd_Basins = DEM_2_LDD_PCRASTER(
#         Path_Input_Dem,
#         Path_Output_Ldd_Basins,
#         Param_PitMaxDepth = Param_PitMaxDepth,
#         Param_RiverDepth  = Param_RiverDepth,
#         Param_NoData      = Param_NoData,
#         outlets           = "edge",
#         River             = [],
#         Idxs_pit          = None)

# N_Pits_Basins, Pits = LDD_2_PITS(Ldd_Basins)

# PITS_TOFILE(crs, Ldd_FlwdirRaster, iXY, Path_Output_Pits)


In [ ]:
# import os as os

# Path_Input_Dem           = os.path.join(Path_Root, Path_Julia, Filename_Input_DemJulia)
# Path_Output_Ldd          = os.path.join(Path_Root, Path_Python, Filename_Output_Ldd)
# Path_Input_River_Geojson = os.path.join(Path_Root, Path_Gis, Filename_Input_RiverShp)
# Path_Output_Pits         = os.path.join(Path_Root, Path_Python, Filename_Output_PitsSubcatchment)

# print("==Ldd Pyflwdir==")
# crs, Ldd, Ldd_FlwdirRaster = DEM_2_LDD_PYFLWDIR(
# 	Path_Input_Dem,
# 	Path_Output_Ldd,
# 	outlets      = "edge",
# 	Idxs_pit     = None)

# iXY, Pits = XY_2_PITS(Ldd_FlwdirRaster, Param_GaugeCoordinate)

# # iiPits_Vector = Ldd_FlwdirRaster.idxs_pit
# # print(iiPits_Vector)

# if not(Ldd_FlwdirRaster.isvalid):
#     print("========= Ldd_Basins_FlwdirRaster not valid ========")

# if Flag_BurnRiver:
# 	River_Geopandas = geopandas.read_file(Path_Input_River_Geojson)
# 	Ldd = DEM_2_LDD_HYDROMT(
# 		Path_Input_Dem,
# 		Path_Output_Ldd,
# 		Param_PitMaxDepth = Param_PitMaxDepth,
# 		Param_RiverDepth  = Param_RiverDepth,
# 		Param_NoData      = Param_NoData,
# 		outlets           = "idxs_pit",
# 		River             = River_Geopandas,
# 		Idxs_pit          = Pits,
# 		Flag_BurnRiver    = Flag_BurnRiver)
# else:
# 	Ldd = DEM_2_LDD_HYDROMT(
# 		Path_Input_Dem,
# 		Path_Output_Ldd,
# 		Param_PitMaxDepth = Param_PitMaxDepth,
# 		Param_RiverDepth  = Param_RiverDepth,
# 		Param_NoData      = Param_NoData,
# 		outlets           = "idxs_pit",
# 		River             = [],
# 		Idxs_pit          = Pits,
# 		Flag_BurnRiver    = Flag_BurnRiver)

# N_Pits, Pits = LDD_2_PITS(Ldd)

# PITS_TOFILE(crs, Ldd_FlwdirRaster, iXY, Path_Output_Pits)

# PLOT_MAPS(Ldd, "Ldd", 1, 9)
### **= SELECTING:** *Bassins =*


In [ ]:
# import geopandas as gpd
# import rasterio
# import pyflwdir


# def FLOWDIRECTION_2_BASINS(
#         Path_Input_Ldd_Basins,
#         Path_Output_Basins_Tiff,
#         Path_Output_Basins_GeoPanda,
#         Name="Basin"):

#     # Read D8 data using rasterio
#     with rasterio.open(Path_Input_Ldd_Basins, "r") as src:
#         FlowDirection = src.read(1)
#         nodata = src.nodata
#         transform = src.transform
#         crs = src.crs
#         latlon = src.crs.is_geographic
#         prof = src.profile

#     Ldd_Basins = pyflwdir.from_array(
#         FlowDirection, ftype="ldd", transform=transform, latlon=latlon, cache=True)

#     # Delineate basins for the whole country
#     Basins = Ldd_Basins.basins()

#     # Write output to raster file
#     prof.update(dtype=Basins.dtype, nodata=False)
#     with rasterio.open(Path_Output_Basins_Tiff, "w", **prof) as src:
#         src.write(Basins, 1)
#     print(Path_Output_Basins_Tiff)

#     # Write output to GeoJSON file
#     Basins_Geopanda = GEOPANDA_VECTORIZE(Basins.astype(
#         np.int32), np.nan, transform=transform, crs=crs, name=Name)
#     Basins_Geopanda.to_file(Path_Output_Basins_GeoPanda,
#                             driver='GeoJSON', mode="w")
#     print(Path_Output_Basins_GeoPanda)

#     return Basins, Basins_Geopanda


In [ ]:
# Masking
Nx, Ny = np.shape(Basins)

Basins_Select = np.ma.empty(shape=(Nx, Ny))

for iX in range(Nx):
   for iY in range(Ny):
      if Basins[iX, iY] == Param_SelectBasin:
         Basins_Select[iX, iY] = 1
      else:
         Basins_Select[iX, iY] = np.nan

PLOT_MAPS(Basins_Select, "Basins", 1, 2, Cmap="viridis")


In [ ]:
# import geopandas


# def READ_SHP(Path_Input_River_Geojson):
#    River_Geopandas = geopandas.read_file(Path_Input_River_Geojson)
#    River_GeoDataFrame = geopandas.GeoDataFrame.from_features(River_Geopandas)
#    return River_GeoDataFrame


### = **PITS:** *Creating pits* =

In [ ]:
def XY_2_PITS(FlwdirRaster, Param_LatLong_Gauge):
   # Returns x, y coordinates of the cell center based on linear cell indices.
   X, Y = np.array([Param_LatLong_Gauge[0]]), np.array([Param_LatLong_Gauge[1]])
   iXY = FlwdirRaster.index( Param_LatLong_Gauge[0], Param_LatLong_Gauge[1])
   Pit = [iXY]

   Xmodel, Ymodel = FlwdirRaster.xy(iXY)
   print("Model coordinate pit = ", [Xmodel, Ymodel])
   print("Given coordinate pit Julia = ", [X+1, Y+1])

   return iXY, Pit
